In [ ]:
import string
import numpy as np
import torch
from PIL import Image
import os
from pickle import dump, load
import numpy as np
from torchvision.models import resnet152
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import string
import numpy as np
from PIL import Image
import os
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.nn.utils.rnn import pad_sequence

# Function to load document file and read the contents inside the file into a string
def load_doc(filename):
    with open(filename, 'r') as f:
        text = f.read()
    return text

# Function to create a dictionary that maps images with a list of 5 captions
def create_descriptions(filename):
    file = load_doc(filename)
    captions = file.split('\n')
    descriptions = {}
    for caption in captions[:-1]:
        img, caption = caption.split('\t')
        if img[:-2] not in descriptions:
            descriptions[img[:-2]] = [caption]
        else:
            descriptions[img[:-2]].append(caption)
    return descriptions

# Function to clean text
def clean_text(captions):
    table = str.maketrans('', '', string.punctuation)
    for img, caps in captions.items():
        for i, img_caption in enumerate(caps):
            img_caption.replace("-", " ")
            desc = img_caption.split()
            # converts to lowercase
            desc = [word.lower() for word in desc]
            # remove punctuation from each token
            desc = [word.translate(table) for word in desc]
            # remove hanging 's and a
            desc = [word for word in desc if len(word) > 1]
            # remove tokens with numbers in them
            desc = [word for word in desc if word.isalpha()]
            # convert back to string
            img_caption = ' '.join(desc)
            captions[img][i] = img_caption
    return captions

# Function to create vocabulary
def create_vocab(descriptions):
    vocab = set()
    for key in descriptions.keys():
        [vocab.update(d.split()) for d in descriptions[key]]
    return vocab

# Function to save preprocessed descriptions into a file named descriptions.txt
def save_descriptions(descriptions, filename):
    lines = []
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + '\t' + desc)
    data = "\n".join(lines)
    with open(filename, "w") as f:
        f.write(data)


In [ ]:
#Set these path according to project folder in you system
dataset_text = path+"Flickr_8k_text"
dataset_images = path+"Flicker8k_Dataset"

#we prepare our text data

filename = dataset_text + "/" + "Flickr8k.token.txt"
filename ='/content/drive/MyDrive/mi/Flickr8k_text/Flickr8k.token.txt'
#loading the file that contains all data
#mapping them into descriptions dictionary img to 5 captions
descriptions = all_img_captions(filename)
print("Length of descriptions =" ,len(descriptions))

#cleaning the descriptions
clean_descriptions = cleaning_text(descriptions)

#building vocabulary
vocabulary = text_vocabulary(clean_descriptions)
print("Length of vocabulary = ", len(vocabulary))

#saving each description to file

save_descriptions(clean_descriptions, "descriptions.txt")
def load_doc(filename):
# Opening the file as read only
with open(filename, 'r') as file:
text = file.read()
return text

This function will create a descriptions dictionary that maps images with a list of 5 captions
def all_img_captions(filename):
file = load_doc(filename)
captions = file.split('\n')
descriptions = {}
for caption in captions[:-1]:
img, caption = caption.split('\t')
if img[:-2] not in descriptions:
descriptions[img[:-2]] = [caption]
else:
descriptions[img[:-2]].append(caption)
return descriptions

Data cleaning- lower casing, removing punctuations and words containing numbers
def cleaning_text(captions):
table = str.maketrans('', '', string.punctuation)
for img, caps in captions.items():
for i, img_caption in enumerate(caps):
      img_caption.replace("-", " ")
      desc = img_caption.split()

      # converts to lowercase
      desc = [word.lower() for word in desc]
      # remove punctuation from each token
      desc = [word.translate(table) for word in desc]
      # remove hanging 's and a 
      desc = [word for word in desc if(len(word) > 1)]
      # remove tokens with numbers in them
      desc = [word for word in desc if(word.isalpha())]
      # convert back to string

      img_caption = ' '.join(desc)
      captions[img][i] = img_caption
return captions

def text_vocabulary(descriptions):
# build vocabulary of all unique words
vocab = set()

for key in descriptions.keys():
    [vocab.update(d.split()) for d in descriptions[key]]

return vocab

In [ ]:
def extract_features(directory):
    model = models.xception(pretrained=True)
    model.eval()
    if torch.cuda.is_available():
        model.cuda()

    transform = transforms.Compose([
        transforms.Resize((299,299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    features = {}
    for img in tqdm(os.listdir(directory)):
        filename = directory + "/" + img
        image = Image.open(filename).convert('RGB')
        image = transform(image)
        image = image.unsqueeze(0)
        if torch.cuda.is_available():
            image = image.cuda()
        
        with torch.no_grad():
            feature = model(image)
        feature = feature.cpu().numpy().squeeze()
        features[img] = feature

    return features

In [ ]:
#This will load the text file in a string and will return the list of image names.
def load_photos(filename):
    file = load_doc(filename)
    photos = file.split("\n")[:-1]
    return photos

'''
This function will create a dictionary that contains captions for each photo from the list of photos.
 We also append the <start> and <end> identifier for each caption.
  We need this so that our LSTM model can identify the starting and ending of the caption.
'''
def load_clean_descriptions(filename, photos): 
    #loading clean_descriptions
    file = load_doc(filename)
    descriptions = {}
    for line in file.split("\n"):

        words = line.split()
        if len(words)<1 :
            continue

        image, image_caption = words[0], words[1:]

        if image in photos:
            if image not in descriptions:
                descriptions[image] = []
            desc = '<start> ' + " ".join(image_caption) + ' <end>'
            descriptions[image].append(desc)

    return descriptions

##  This function will return a dictionary for image names and their feature vector which we have previously extracted from the Xception model.
def load_features(photos):
    #loading all features
    all_features = load(open("/content/drive/MyDrive/mi/new/features.p","rb"))
    #selecting only needed features
    features = {k:all_features[k] for k in photos}
    return features



In [ ]:
# filename = dataset_text + "/" + "Flickr_8k.trainImages.txt"
filename = '/content/drive/MyDrive/mi/Flickr8k_text/Flickr_8k.trainImages.txt'

#train = loading_data(filename)
train_imgs = load_photos(filename)
train_descriptions = load_clean_descriptions("/content/drive/MyDrive/mi/new/descriptions.txt", train_imgs)
train_features = load_features(train_imgs)

In [ ]:
def dict_to_list(descriptions):
    all_desc = []
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

from torchtext.data.utils import get_tokenizer

def create_tokenizer(descriptions):
    desc_list = dict_to_list(descriptions)
    tokenizer = get_tokenizer('basic_english')
    tokenizer.fit_on_texts(desc_list)
    return tokenizer

In [ ]:
def max_length(descriptions):
    desc_list = dict_to_list(descriptions)
    return max(len(d.split()) for d in desc_list)

In [ ]:
max_length = max_length(descriptions)
max_length

In [ ]:
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn

# custom dataset class
class ImageCaptionDataset(Dataset):
    def __init__(self, descriptions, features, tokenizer, max_length):
        self.descriptions = descriptions
        self.features = features
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.vocab_size = len(tokenizer.word_index) + 1

    def __len__(self):
        return len(self.descriptions)

    def __getitem__(self, idx):
        key = list(self.descriptions.keys())[idx]
        # retrieve photo features
        feature = torch.Tensor(self.features[key][0])
        input_image, input_sequence, output_word = self.create_sequences(self.tokenizer, self.max_length, self.descriptions[key], feature)
        return input_image, input_sequence, output_word

    def create_sequences(self, tokenizer, max_length, desc_list, feature):
        X1, X2, y = [], [], []
        # walk through each description for the image
        for desc in desc_list:
            # encode the sequence
            seq = tokenizer.texts_to_sequences([desc])[0]
            # split one sequence into multiple X,y pairs
            for i in range(1, len(seq)):
                # split into input and output pair
                in_seq, out_seq = seq[:i], seq[i]
                # pad input sequence
                in_seq = nn.utils.rnn.pad_sequence([torch.LongTensor(in_seq)], batch_first=True, padding_value=0)
                # encode output sequence
                out_seq = torch.zeros(self.vocab_size)
                out_seq[out_seq == 0] = -1e9 # setting the probability of padding token to be very low
                out_seq[out_seq == 0] = 0 # setting the probability of all other tokens to be zero
                out_seq[out_seq == out_seq[out_seq != 0][out_seq[out_seq != 0].argmax()]] = 1 # setting the probability of the actual output token to be one
                # store
                X1.append(feature)
                X2.append(in_seq.squeeze(0))
                y.append(out_seq)
        return torch.stack(X1), torch.stack(X2), torch.stack(y)

# dataloader for the dataset
def get_loader(descriptions, features, tokenizer, max_length, batch_size, shuffle=True, num_workers=2):
    dataset = ImageCaptionDataset(descriptions, features, tokenizer, max_length)
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
    return dataloader


In [ ]:
#You can check the shape of the input and output for your model
[a,b],c = next(data_generator(train_descriptions, features, tokenizer, max_length))
a.shape, b.shape, c.shape
# ((47, 2048), (47, 32), (47, 7577))

In [ ]:
class CaptioningModel(nn.Module):
def init(self, vocab_size, max_length):
    super(CaptioningModel, self).init()
    # features from the CNN model squeezed from 2048 to 256 nodes
    self.fe1 = nn.Dropout(0.5)
    self.fe2 = nn.Linear(2048, 256)
    # LSTM sequence model
    self.se1 = nn.Embedding(vocab_size, 256)
    self.se2 = nn.Dropout(0.5)
    self.se3 = nn.LSTM(256, 256, batch_first=True)
    # Merging both models
    self.decoder1 = nn.Linear(256, 256)
    self.decoder2 = nn.Linear(256, vocab_size)
def forward(self, img_features, captions):
    img_features = self.fe1(img_features)
    img_features = self.fe2(img_features)
    captions = self.se1(captions)
    captions = self.se2(captions)
    captions, _ = self.se3(captions)
    decoder_input = img_features + captions
    decoder_input = self.decoder1(decoder_input)
    decoder_input = nn.functional.relu(decoder_input)
    outputs = self.decoder2(decoder_input)

    return outputs

model = CaptioningModel(vocab_size, max_length)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(model)

In [ ]:
# train our model
print('Dataset: ', len(train_imgs))
print('Descriptions: train=', len(train_descriptions))
print('Photos: train=', len(train_features))
print('Vocabulary Size:', vocab_size)
print('Description Length: ', max_length)

model = define_model(vocab_size, max_length)
epochs = 10
steps = len(train_descriptions)
# making a directory models to save our models
os.mkdir("models")
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    model.fit_generator(generator, epochs=1, steps_per_epoch= steps, verbose=1)
    model.save("models/model_" + str(i) + ".h5")

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import argparse


# ap = argparse.ArgumentParser()
# ap.add_argument('-i', '--image', required=False, help="Image Path")
# args = vars(ap.parse_args())
# img_path = args['image']


img_path = '/content/drive/MyDrive/mi/Flicker8k_Dataset/10815824_2997e03d76.jpg'

#extractsfeatures of the test image

def extract_features(filename, model):
        try:
            image = Image.open(filename)

        except:
            print("ERROR: Couldn't open image! Make sure the image path and extension is correct")
        image = image.resize((299,299))
        image = np.array(image)
        # for images that has 4 channels, we convert them into 3 channels
        if image.shape[2] == 4: 
            image = image[..., :3]
        image = np.expand_dims(image, axis=0)
        image = image/127.5
        image = image - 1.0
        feature = model.predict(image)
        return feature

def word_for_id(integer, tokenizer):
  for word, index in tokenizer.word_index.items():
      if index == integer:
          return word
  return None

# generates the caption for the given image
def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo,sequence], verbose=0)
        pred = np.argmax(pred)
        word = word_for_id(pred, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text


#path = 'Flicker8k_Dataset/111537222_07e56d5a30.jpg'
max_length = 32
tokenizer = load(open("/content/drive/MyDrive/mi/new/tokenizer.p","rb"))
model = load_model('/content/drive/MyDrive/mi/new/model_9.h5')
xception_model = Xception(include_top=False, pooling="avg")

photo = extract_features(img_path, xception_model)
img = Image.open(img_path)

description = generate_desc(model, tokenizer, photo, max_length)
print("\n\n")
print(description)
plt.imshow(img)

In [ ]:
img_path = '/content/drive/MyDrive/mi/Flicker8k_Dataset/241345522_c3c266a02a.jpg'
image = Image.open(img_path)

photo = extract_features(img_path, xception_model)
img = Image.open(img_path)

description = generate_desc(model, tokenizer, photo, max_length)
print("\n\n")
print(description)
plt.imshow(img)

In [ ]:
img_path = '/content/drive/MyDrive/mi/Flicker8k_Dataset/481827288_a688be7913.jpg'
image = Image.open(img_path)

photo = extract_features(img_path, xception_model)
img = Image.open(img_path)

description = generate_desc(model, tokenizer, photo, max_length)
print("\n\n")
print(description)
plt.imshow(img)



In [ ]:
img_path = '/content/drive/MyDrive/mi/Flicker8k_Dataset/2511019188_ca71775f2d.jpg'
image = Image.open(img_path)

photo = extract_features(img_path, xception_model)
img = Image.open(img_path)

description = generate_desc(model, tokenizer, photo, max_length)
print("\n\n")
print(description)
plt.imshow(img)


In [ ]:
img_path = '/content/drive/MyDrive/mi/Flicker8k_Dataset/3150742439_b8a352e1e0.jpg'
image = Image.open(img_path)

photo = extract_features(img_path, xception_model)
img = Image.open(img_path)

description = generate_desc(model, tokenizer, photo, max_length)
print("\n\n")
print(description)
plt.imshow(img)

In [ ]:
# creates a dictionary of all test images along with their reference captions for calculating bleu score
def load_clean_descriptions2(filename, photos): 
    #loading clean_descriptions
    file = load_doc(filename)
    descriptions = {}
    for line in file.split("\n"):

        words = line.split()
        if len(words)<1 :
            continue

        image, image_caption = words[0], words[1:]

        if image in photos:
            if image not in descriptions:
                descriptions[image] = []
            desc = " ".join(image_caption)
            # desc=image_caption
            descriptions[image].append(desc)

    return descriptions

In [ ]:
# filename = dataset_text + "/" + "Flickr_8k.testImages.txt"
filename = '/content/drive/MyDrive/mi/Flickr8k_text/Flickr_8k.testImages.txt'

#test = loading_data(filename)
test_imgs = load_photos(filename)
# test_descriptions = all_img_captions("/content/drive/MyDrive/mi/Flickr8k_text/Flickr8k.token.txt")
test_descriptions = load_clean_descriptions2("/content/drive/MyDrive/mi/Flickr8k_text/Flickr8k.token2.txt", test_imgs)
# train_features = load_features(train_imgs)

In [ ]:
 # evaluate the skill of the model using bleu score
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	# step over the whole set
	for key, desc_list in descriptions.items():
		# generate description
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		# store actual and predicted
		references = [d.split() for d in desc_list]
		actual.append(references)
		predicted.append(yhat.split())
	# calculate BLEU score
	print('BLEU-1: %f' % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [ ]:
# Creates a disctionaly photos haveing images and their extracted fesautes
img_path = '/content/drive/MyDrive/mi/Flicker8k_Dataset/'
photos={}
for i in range(len(test_imgs)):
  img_path+=test_imgs[i]
  # print(img_path)
  # image = Image.open(img_path)
  photo = extract_features(img_path, xception_model)
  photos[test_imgs[i]] = photo
  print(i)
  img_path = '/content/drive/MyDrive/mi/Flicker8k_Dataset/'

In [ ]:
evaluate_model(model, test_descriptions, photos, tokenizer, max_length)